In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.preprocessing as sp
import pickle
import matplotlib.pyplot as plt
import scipy
import os
import time
from datetime import date
# import dask.dataframe as dd
# import dask.config
# dask.config.set({"distributed.scheduler.allowed-failures": 10})
today = date.today()
import os
import sys
sys.path.insert(0, '/home/ubuntu/workspace_SingleCell/SingleCell_Morphological_Analysis/') 
from singlecell.read import read_single_cell_sql
from singlecell.preprocess import handle_nans, extract_cpfeature_names,find_highly_correlated_features
from singlecell.visualize import visualize_n_SingleCell, cluster
from singlecell.process import statistical_tests,precision_recall
from singlecell.preprocess.filter_out_edge_single_cells import edgeCellFilter
from singlecell.save.save_pandas_dfs import saveDF_to_CSV_GZ_no_timestamp
from singlecell.preprocess.control_for_cellcount import control_feature_y_for_variable_x
from singlecell.process.replicate_correlation import replicate_null_corr_coefs
from singlecell.process import normalize_funcs


DEBUG:jupyter_black:config: {'line_length': 79, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

In [8]:
########################## Project root directory and path to results ########################
mito_project_root_dir = "/home/ubuntu/bucket/projects/2016_08_01_RadialMitochondriaDistribution_donna/"
save_results_dir = mito_project_root_dir + "/workspace/results/jump_fq/"

with open(
    "/home/ubuntu/workspace_periscope/2022_PERISCOPE/common_files/annotated_gene_sets.json"
) as f:
    gene_set_dict = json.load(f)

In [3]:
ls /home/ubuntu/workspace_periscope/2022_PERISCOPE/common_files

annotated_gene_sets.json  CORUM_humanComplexes.txt  STRING_data.csv.gz
Barcodes.csv              README.md


In [5]:
import pandas as pd
from sqlalchemy import create_engine
from functools import reduce
import gc


def read_per_well_data(
    input_data_dir,
    annot,
    prof_workspace_folder_name="profiles",
    fformat=".parquet",
):
    batches = annot["Batch"].unique()

    df_agg_all_batches_ls = []
    for b in batches:
        print(b)
        #         if "Metadata_Source" in annot.columns:
        source_str = annot.loc[
            annot["Batch"] == b, "Metadata_Source"
        ].unique()[0]
        #             print(source_str)
        profile_path = (
            input_data_dir
            + source_str
            + "/workspace/"
            + prof_workspace_folder_name
            + "/"
        )
        #         else:
        #             profile_path = input_data_dir + "/workspace/profiles/"

        df_sag_ls = []
        plates_exist = os.listdir(profile_path + b)
        plates_meta = annot.loc[annot["Batch"] == b, "Metadata_Plate"].unique()
        plates = set(plates_meta) & set(plates_exist)
        for p in plates:
            print(p)

            fileName = profile_path + b + "/" + p + "/" + p + fformat
            #             print(fileName)
            if os.path.exists(fileName):
                if fformat == ".parquet":
                    sc_df = pd.read_parquet(fileName)
                elif fformat in [".csv", ".csv.gz"]:
                    sc_df = pd.read_csv(fileName)

                #         per_site_aggregate=sc_df.groupby(['Metadata_Well','Metadata_Site']).mean()[feature_list+['Count_Cells']].reset_index()
                sc_df["Metadata_Batch"] = b
                sc_df["Metadata_Plate"] = p
                df_sag_ls.append(sc_df)
                del sc_df
                gc.collect()
            else:
                print(fileName, " not exists")

        if df_sag_ls:
            df_sag = pd.concat(df_sag_ls, axis=0)
            df_agg_all_batches_ls.append(df_sag)

    df_agg_all_batches = pd.concat(df_agg_all_batches_ls, axis=0)
    return df_agg_all_batches

In [6]:
lincs_meta_cols = [
    "Metadata_broad_sample",
    "Metadata_dose_recode",
    "Metadata_pert_id",
    "Metadata_pert_mfc_id",
    "Metadata_InChIKey14",
    "Metadata_pert_type",
    "Metadata_moa",
    "Metadata_target",
    "Metadata_pert_id_dose",
    "Metadata_pert_name",
]

# lincs_meta_cols=['Metadata_broad_sample','Metadata_dose_recode','Metadata_pert_id','Metadata_pert_mfc_id',\
# 'Metadata_InChIKey14','Metadata_pert_type','Metadata_pert_id_dose']

cdrp_meta_cols = [
    "Metadata_broad_sample",
    "Metadata_mmoles_per_liter2",
    "Metadata_pert_id",
    "Metadata_Sample_Dose",
    "Metadata_moa",
]
jumporf_meta_cols = ["Symbol", "broad_sample"]
jumpcrispr_meta_cols = ["Metadata_NCBI_Gene_ID", "Metadata_Symbol"]
jumpcompound_meta_cols = ["Metadata_InChIKey", "Metadata_InChI"]
taorf_meta_cols = [
    "Metadata_gene_name",
    "Metadata_pert_name",
    "Metadata_broad_sample",
    "Metadata_moa",
]


# jump_orf_params={'profiles_path':"/home/ubuntu/jumpbucket/projects/2021_04_26_Production/workspace/backend/",\
#                  'meta_cols':jumporf_meta_cols,\
#                  'pert_col':'broad_sample',\
#                  'target_features_list':target_features_list_orf_cdrp
#                 }

jump_orf_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0016-jump/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".parquet",
    "meta_cols": jumporf_meta_cols,
    "pert_col": "Metadata_JCP2022",
    "untreated_key_val": ["Metadata_pert_type", "negcon"],
}

cdrp_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0012-wawer-bioactivecompoundprofiling/broad/workspace/backend/",
    "meta_cols": cdrp_meta_cols,
    "pert_col": "Metadata_Sample_Dose",
}
# /home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/workspace/backend/
# https://cellpainting-gallery.s3.amazonaws.com/cpg0004-lincs/broad/workspace/backend/2016_04_01_a549_48hr_batch1/SQ00014812
lincs_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0004-lincs/",
    "prof_workspace_folder_name": "backend",
    "pformat": ".csv",
    "meta_cols": lincs_meta_cols,
    "pert_col": "Metadata_pert_id_dose",
    "untreated_key_val": ["Metadata_pert_type", "control"],
}

jump_crispr_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0016-jump/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".parquet",
    "meta_cols": jumpcrispr_meta_cols,
    "pert_col": "Metadata_JCP2022",
    "untreated_key_val": ["Metadata_Symbol", "non-targeting"],
}

jump_compound_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0016-jump/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".parquet",
    "meta_cols": jumpcompound_meta_cols,
    "pert_col": "Metadata_JCP2022",
    "untreated_key_val": ["Metadata_JCP2022", "JCP2022_999999"],
}

ta_orf_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0017-rohban-pathways/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".csv.gz",
    "meta_cols": taorf_meta_cols,
    "pert_col": "Metadata_broad_sample",
    "untreated_key_val": ["Metadata_pert_type", "Untreated"],
}

ds_info_dict = {
    "jump_orf": jump_orf_params,
    "CDRP": cdrp_params,
    "lincs": lincs_params,
    "lincs_g": lincs_params,
    "jump_crispr": jump_crispr_params,
    "jump_compound": jump_compound_params,
    "taorf": ta_orf_params,
}
# 'broad_sample', 'pert_type', 'control_type'

# results=annot[['Symbol','broad_sample', 'pert_type', 'control_type']].drop_duplicates().reset_index(drop=True)

# dataset='CDRP';dataset_meta_hue='Metadata_moa'
# dataset='lincs';dataset_meta_hue='Metadata_moa'
# dataset='jump_orf';dataset_meta_hue='Symbol'

In [9]:
datasets = ["jump_orf", "jump_crispr"]

symbol_col = {
    "jump_crispr": "Metadata_Symbol",
    "jump_orf": "Metadata_Symbol",
    "taorf": "Metadata_gene_name",
}

df_ls = []
# names=[]
ds_symbols = {}
ds_features = {}
for dataset in datasets:
    #     names.append(dataset)
    annot = pd.read_csv(
        mito_project_root_dir
        + "/workspace/metadata/preprocessed/annot_"
        + dataset
        + ".csv",
        dtype={"Metadata_Plate": str},
    )
    if "Metadata_Source" not in annot.columns:
        annot["Metadata_Source"] = "broad"
    # target_features_list = ds_info_dict[dataset]["target_features_list"]
    # sources = ['source_5']
    sources = annot["Metadata_Source"].unique()
    for si in sources:
        file_path = (
            save_results_dir
            + "/preprocessed_data/"
            + dataset
            + "_df_rep_level_scaled_"
            + si
            + ".csv"
        )
        if os.path.exists(file_path):
            df_rep_level_scaled = pd.read_csv(file_path)

            ds_symbols[dataset] = (
                df_rep_level_scaled[symbol_col[dataset]].unique().tolist()
            )

            (
                cp_features,
                cp_features_analysis_0,
            ) = extract_cpfeature_names.extract_cpfeature_names(
                df_rep_level_scaled
            )

            ds_features[dataset] = cp_features_analysis_0

            #             df_rep_level_scaled['']

            #         batch_names_cols = DS_X_featureQ_rank_df.columns[
            #             DS_X_featureQ_rank_df.columns.str.contains("^average_std_score_")
            #         ].tolist()
            df_ls.append(df_rep_level_scaled)

In [10]:
overlap_symbols = list(
    set.intersection(*(set(values) for values in ds_symbols.values()))
)

overlap_features = list(
    set.intersection(*(set(values) for values in ds_features.values()))
)

print(
    "overlap_symbols size: ",
    len(overlap_symbols),
    "overlap_features size: ",
    len(overlap_features),
)

overlap_symbols size:  5245 overlap_features size:  3378


In [ ]:
# Function to calculate percentage of significant number of features from each channel
def df_maker(gene_group, df_genes):
    mito, cona, wga, dapi, phal = df_genes.Mito.mean(), df_genes.ConA.mean(), df_genes.WGA.mean(), df_genes.DAPI.mean(), df_genes.Phalloidin.mean()
    sum_all = df_genes.Sum.mean()
    data.loc[gene_group,'Mito'] = mito/sum_all*100
    data.loc[gene_group,'ConA'] = cona/sum_all*100
    data.loc[gene_group,'WGA'] = wga/sum_all*100
    data.loc[gene_group,'DAPI'] = dapi/sum_all*100
    data.loc[gene_group,'Phalloidin'] = phal/sum_all*100
    
    return data

# Organize the data into a dataframe
gene_groups = ['Vacuolar-type ATPase','Protein O-mannosylation','Outer Mitochondrial Membrane Protein Complex',
               'Cortical Cytoskeleton','DNA Polymerase Complex']
data = pd.DataFrame(index=gene_groups)

for gene_group in gene_groups:
    gene_list = gene_set_dict[gene_group]
    df_genes = df_sig_feature.loc[gene_list]
    df_genes = df_genes.reset_index().query('index in @expressed_gene_list').query('index in @hit_list')
    data = df_maker(gene_group, df_genes)

data

In [ ]:
# Plot fraction of features per compartments for Fig 2E
for group in list(data.index):
    print(group)
    group_name = group.replace(' ','_')
    names = ['Mito','ER', 'Golgi/\nMembrane', 'DNA','Actin']
    size = data.loc[group].values
 
    # Create a circle at the center of the plot
    my_circle = plt.Circle( (0,0), 0.7, color='white')

    # Custom wedges
    plt.pie(size, 
            labels=names, 
            wedgeprops = { 'linewidth' : 7, 'edgecolor' : 'white' },
            colors = ['lightcoral','orchid','lightblue','cornflowerblue','lightgreen'],
            autopct='%1.0f%%', 
            pctdistance=0.83)
    p = plt.gcf()
    p.gca().add_artist(my_circle)
    plt.savefig(os.path.join(output_folder,'figure_panels',f'Fig2E_A549_fraction_features_per_compartments_{group_name}.png'), 
                dpi=300,
                facecolor='w', 
                edgecolor='w',
                bbox_inches='tight')
    plt.show()